In [ ]:
# import necessary libraries and reload the data to continue analysis
import pandas as pd

# Load the data
file_path = '/content/sumupCaseStudy11.csv'
data = pd.read_csv(file_path)

# Convert 'CREATED_AT' to datetime
data['CREATED_AT'] = pd.to_datetime(data['CREATED_AT'])

# Sort data by MERCHANT_ID, REASON, and CREATED_AT to process resolution based on 7-day criteria
data = data.sort_values(by=['MERCHANT_ID', 'REASON', 'CREATED_AT'])

# Calculate the difference in days between consecutive interactions within each merchant-topic group
data['TIME_DIFF'] = data.groupby(['MERCHANT_ID', 'REASON'])['CREATED_AT'].diff().dt.days

# Mark cumulative 7-day windows by grouping interactions as resolved if they occur within 7 days
data['FOLLOW_UP_WITHIN_7_DAYS'] = data['TIME_DIFF'] <= 7

# Filter rows with agent companies
data_with_agent = data.dropna(subset=['AGENT_COMPANY'])

# Group by agent and merchant-topic pairs to count cases that meet resolution criteria (sequences within 7 days)
resolved_cases_by_agent = data_with_agent[data_with_agent['FOLLOW_UP_WITHIN_7_DAYS']].groupby(
    ['AGENT_COMPANY', 'MERCHANT_ID', 'REASON']).size().reset_index(name='RESOLVED_COUNT')

# Calculate performance metrics (average handling time, resolution rate) for each agent company
agent_performance = data_with_agent.groupby('AGENT_COMPANY').agg(
    AVG_HANDLING_TIME=('TOTAL_HANDLING_TIME_SECONDS', 'mean'), # Average handling time
    RESOLVED_CASES=('STATUS', lambda x: (x == 'Resolved').sum()),  # Resolved cases count
    TOTAL_CASES=('STATUS', 'count')  # Total cases handled
).reset_index()

# Calculate resolution rate as percentage of resolved cases
agent_performance['RESOLUTION_RATE'] = (agent_performance['RESOLVED_CASES'] / agent_performance['TOTAL_CASES']) * 100

# Sort by resolution rate and handling time for insight on best-performing agents
agent_performance_sorted = agent_performance.sort_values(by=['RESOLUTION_RATE', 'AVG_HANDLING_TIME'], ascending=[False, True])
agent_performance_sorted.head(10)


In [ ]:
# Given data for monthly investment and working hours
monthly_investment = 0.5e6  # €0.5M
hours_per_week = 40  # Each agent works 40 hours per week
weeks_per_month = 4  # Approximately 4 weeks in a month
total_hours_per_month = hours_per_week * weeks_per_month

# Calculate hourly cost per agent
hourly_cost_per_agent = monthly_investment / total_hours_per_month

# Calculate cost per interaction based on channel type (considering concurrency for chat and exclusivity for calls)
data_with_agent['INTERACTION_COST'] = data_with_agent.apply(
    lambda row: hourly_cost_per_agent * (row['TOTAL_HANDLING_TIME_SECONDS'] / 3600) / (3 if row['CHANNEL'] == 'chat' else 1),
    axis=1
)

# Sum up costs by channel to determine the overall spending per channel
channel_cost_summary = data_with_agent.groupby('CHANNEL')['INTERACTION_COST'].sum().reset_index()

# Display the total cost per channel
channel_cost_summary.sort_values(by='INTERACTION_COST', ascending=False)
